In [1]:
import pandas as pd
from ANNbuilder import build_network
from pso import pso_optimization
from loss import mse_loss
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv(
    "/Users/ilya/Desktop/GitHub_Repositories/HW_University/Data_Mining/datasets/heart.csv")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
data.shape

(1025, 14)

In [4]:
data.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [5]:
data["target"].value_counts()

target
1    526
0    499
Name: count, dtype: int64

In [6]:
X = data.drop("target", axis=1).values
y = data["target"].values.reshape(-1, 1)

In [7]:
X[:5]

array([[ 52. ,   1. ,   0. , 125. , 212. ,   0. ,   1. , 168. ,   0. ,
          1. ,   2. ,   2. ,   3. ],
       [ 53. ,   1. ,   0. , 140. , 203. ,   1. ,   0. , 155. ,   1. ,
          3.1,   0. ,   0. ,   3. ],
       [ 70. ,   1. ,   0. , 145. , 174. ,   0. ,   1. , 125. ,   1. ,
          2.6,   0. ,   0. ,   3. ],
       [ 61. ,   1. ,   0. , 148. , 203. ,   0. ,   1. , 161. ,   0. ,
          0. ,   2. ,   1. ,   3. ],
       [ 62. ,   0. ,   0. , 138. , 294. ,   1. ,   1. , 106. ,   0. ,
          1.9,   1. ,   3. ,   2. ]])

In [8]:
# Data separation and scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# Определение архитектуры и активационных функций сети
architecture = [13, 26, 26, 1]  # Архитектура сети
activations = ['relu', "relu", 'logistic']  # Активационные функции

# Нужно использовать функцию build_network для создания сети
network = build_network(architecture, activations)

# Получение размерности вектора весов сети
dimensions = network.total_weights()

# Правильный вызов pso_optimize с созданной сетью и параметрами
best_weights = pso_optimization(
    num_particles=100,
    num_iterations=300,
    loss_func=mse_loss,
    network=network,
    train_data=X_train_scaled,
    train_labels=y_train,
    dimensions=dimensions)


# Установка лучших весов в сеть
network.set_weights(best_weights)

# Тестирование сети
output = network.forward(X_test_scaled)
predictions = (output > 0.5).astype(int)

# Вывод результатов
print(classification_report(y_test, predictions))

Iteration 25/300, Current Best Loss: 0.1573170731707317
Iteration 50/300, Current Best Loss: 0.14268292682926828
Iteration 75/300, Current Best Loss: 0.13658536585365855
Iteration 100/300, Current Best Loss: 0.13048780487804879
Iteration 125/300, Current Best Loss: 0.12560975609756098
Iteration 150/300, Current Best Loss: 0.11829268292682926
Iteration 175/300, Current Best Loss: 0.10975609756097561
Iteration 200/300, Current Best Loss: 0.10121951219512196
Iteration 225/300, Current Best Loss: 0.0975609756097561
Iteration 250/300, Current Best Loss: 0.09634146341463415
Iteration 275/300, Current Best Loss: 0.09390243902439024
Iteration 300/300, Current Best Loss: 0.09390243902439024
Final Best Loss: 0.09390243902439024 on Iteration: 252
              precision    recall  f1-score   support

           0       0.89      0.83      0.86       102
           1       0.85      0.90      0.87       103

    accuracy                           0.87       205
   macro avg       0.87      0.87   